In [2]:
import numpy as np
import pandas as pd
import torch
from allennlp.predictors.predictor import Predictor

In [3]:
test_df = pd.read_csv('test.csv')[['Event','Xintent','Xemotion','Otheremotion']]
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
for col in ['Xintent','Xemotion','Otheremotion']:
    test_df[col] = test_df[col].apply(str2list)

In [4]:
def clear_word(w):
    x = w
    if x[:3] == 'is ' or x[:3] == 'to ':
        x = x[3:]
    if x[:3] == 'be ':
        x = x[3:]
    if x[len(x) - 1] == '.':\
        x = x[:len(x)-1]
    return x

In [5]:
phrase2oemotion = {}
phrase2xemotion = {}
phrase2xintent = {}
res_xi = []
res_xe = []
res_oe = []
set_xi = set()
set_xe = set()
set_oe = set()

phrase_set = set([x for x in test_df.Event])

for phrase in phrase_set:
    for x in test_df[test_df.Event == phrase].Xintent:
        res_xi += x
    phrase2xintent[phrase] = set()
    for x in set(res_xi):
        if x != '':
            phrase2xintent[phrase].add(clear_word(x))
    res_xi.clear()
    
    for x in test_df[test_df.Event == phrase].Xemotion:
        res_xe += x
    phrase2xemotion[phrase] = set()
    for x in set(res_xe):
        if x != '':
            phrase2xemotion[phrase].add(clear_word(x))
    res_xe.clear()
    
    for x in test_df[test_df.Event == phrase].Otheremotion:
        res_oe += x
    phrase2oemotion[phrase] = set()
    for x in set(res_oe):
        if x != '':
            phrase2oemotion[phrase].add(clear_word(x))
    res_oe.clear()

In [6]:
p_small = Predictor.from_path("ser_dir_original_model/model.tar.gz")
p_large = Predictor.from_path("ser_dir_large_model/model.tar.gz")
p_mid = Predictor.from_path("ser_dir_5e_model/model.tar.gz")
p_orig = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/event2mind-2018.10.26.tar.gz")

ConfigurationError: 'spacy not in acceptable choices for dataset_reader.source_tokenizer.type: [\'word\', \'pretrained_transformer\', \'character\']. You should either use the --include-package flag to make sure the correct module is loaded, or use a fully qualified class name in your config file like {"model": "my_module.models.MyModel"} to have it imported automatically.'

In [9]:
p_orig = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/event2mind-2018.10.26.tar.gz")
p_new_xi_2e = Predictor.from_path("xi_2e_new_model/model.tar.gz")
p_new_xi_10e = Predictor.from_path("xi_10e_new_model/model.tar.gz")
p_birnn300_xi_2e = Predictor.from_path("xi_2e_birnn300_model/model.tar.gz")
p_birnn300_xi_10e = Predictor.from_path("xi_10e_birnn300_model/model.tar.gz")
p_cnn_xi_2e = Predictor.from_path("xi_2e_cnn_model/model.tar.gz")

In [17]:
def intents(predictor, p):
    return set([" ".join(react) for react in predictor.predict(source=p)["xintent_top_k_predicted_tokens"]])
    
def xreact(predictor, p):
    return set([" ".join(react) for react in predictor.predict(source=p)["xreact_top_k_predicted_tokens"]])
    
def oreact(predictor, p):
    return set([" ".join(react) for react in predictor.predict(source=p)["oreact_top_k_predicted_tokens"]])

def predictor_info(predictor, p):
    print("Intent: ", intents(predictor, p))
    print("Xreact: ", xreact(predictor, p))
    print("Oreact: ", oreact(predictor, p))
    print()

def compare_orig_with(predictor, p):
    print("Intent: ", intents(p_orig, p)&intents(predictor, p))
    print("Xreact: ", xreact(p_orig, p)&xreact(predictor, p))
    print("Oreact: ", oreact(p_orig, p)&oreact(predictor, p))
    print()

def experiment(p):
    print("--------------------------------------------------------------")
    print("Фраза: ", p)
    print()
    
    print("Модель из статьи")
    predictor_info(p_orig, p)
    
    print("Модель BiRNN100 на 2 эпохах")
    predictor_info(p_new_xi_2e, p)
    
    print("Модель BiRNN100 на 10 эпохах")
    predictor_info(p_new_xi_10e, p)
    
    print("Модель BiRNN300 на 2 эпохах")
    predictor_info(p_birnn300_xi_2e, p)
    
    print("Модель BiRNN300 на 10 эпохах")
    predictor_info(p_birnn300_xi_10e, p)
    
    print("Модель CNN на 2 эпохах")
    predictor_info(p_cnn_xi_2e, p)
    
    print("Пересечение ответов оригинальной модели с BiRNN100")
    compare_orig_with(p_new_xi_10e, p)

    print("Пересечение ответов оригинальной модели с BiRNN300")
    compare_orig_with(p_birnn300_xi_10e, p)

    print("Пересечение ответов оригинальной модели с CNN")
    compare_orig_with(p_cnn_xi_2e, p)

In [16]:
phrases = ["PersonX takes a nap",
     "PersonX punches PersonY",
     "PersonX writes a code",
     "PersonX takes part in an intership",
     "PresonX codes all night long",
     "PersonX has tonns of deadlines",
     "PersonX suggested PersonY to work together"]

for p in phrases:
    experiment(p)

--------------------------------------------------------------
Фраза:  PersonX takes a nap

Модель из статьи
Intent:  {'none', 'rest', 'get some exercise', 'rested', 'have fun', 'get out of town', 'sleep', 'relax', 'get some rest', 'get out of the house'}
Xreact:  {'refreshed', 'relaxed', 'rested', 'tired', 'sleepy', 'happy', 'relieved', 'comfortable', 'calm', 'awake'}
Oreact:  {'none', 'relaxed', 'tired', 'grateful', 'happy', 'comfortable', 'glad', 'annoyed', 'helpful', 'appreciative'}

Модель BiRNN100 на 2 эпохах
Intent:  {'none', 'eat', 'have a good time', 'get a good job', 'have fun', 'make money', 'learn', 'relax', 'get a good grade', 'have fun .'}
Xreact:  {'excited', 'none', 'satisfied', 'happy', 'accomplished', 'relieved', 'good', 'proud', 'none .', 'helpful'}
Oreact:  {'angry', 'loved', 'none', 'sad', 'grateful', '@@UNKNOWN@@', 'happy', 'thankful', 'none .', 'annoyed'}

Модель BiRNN100 на 10 эпохах
Intent:  {'get away', 'rest', 'get some exercise', 'healthy', 'get home', 'take

Intent:  {'inform', 'communicate'}
Xreact:  {'accomplished', 'happy', 'satisfied'}
Oreact:  {'none', 'happy'}

Пересечение ответов оригинальной модели с BiRNN300
Intent:  {'@@UNKNOWN@@'}
Xreact:  {'accomplished', 'excited', 'happy', 'satisfied'}
Oreact:  {'none', 'happy'}

Пересечение ответов оригинальной модели с CNN
Intent:  {'none'}
Xreact:  {'accomplished', 'excited', 'happy', 'satisfied'}
Oreact:  {'none', 'happy'}

--------------------------------------------------------------
Фраза:  PersonX takes part in an intership

Модель из статьи
Intent:  {'a good friend', 'none', 'responsible', 'have a good time', 'a good person', 'have fun', 'the thing', 'a better person', 'do something', 'helpful'}
Xreact:  {'excited', 'good about themselves', 'satisfied', 'responsible', 'happy', 'accomplished', 'relieved', 'good', 'proud', 'helpful'}
Oreact:  {'none', 'satisfied', 'grateful', 'happy', 'relieved', 'thankful', 'impressed', 'good', 'glad', 'pleased'}

Модель BiRNN100 на 2 эпохах
Intent:  

Xreact:  {'excited', 'none', 'satisfied', 'happy', 'accomplished', 'none ,', 'none and happy', 'none .', 'eager', 'helpful'}
Oreact:  {'none', 'loved', 'sad', 'grateful', 'scared', 'happy', 'thankful', 'none for', 'appreciated', 'none .'}

Модель CNN
Intent:  {'none', 'have a good time', 'get a good job', 'healthy', 'have fun', 'get a good time', 'happy', 'win', 'get to work', 'none .'}
Xreact:  {'excited', 'none', 'none and', 'satisfied', 'none none', 'happy', 'accomplished', 'good', 'proud', 'none .'}
Oreact:  {'angry', 'loved', 'none', 'also none', 'none none', 'happy', 'thankful', 'none also', 'none .', 'annoyed'}

Пересечение ответов оригинальной модели с BiRNN100
Intent:  {'@@UNKNOWN@@', 'none', 'free'}
Xreact:  {'satisfied'}
Oreact:  {'none', 'happy'}

Пересечение ответов оригинальной модели с BiRNN300
Intent:  {'@@UNKNOWN@@', 'have fun', 'none', 'have a good time'}
Xreact:  {'excited', 'happy', 'satisfied'}
Oreact:  {'none', 'happy'}

Пересечение ответов оригинальной модели с C